In [31]:
import pyscreenshot as ImageGrab
import time as time
import tkinter as tk
from PIL import ImageGrab
import os

# Folder to save images
images_folder = "captured_images/3/"
if not os.path.exists(images_folder):
    os.makedirs(images_folder)

# Set up the main window
root = tk.Tk()
root.title("Draw and Capture")

# Set up the canvas for drawing
canvas = tk.Canvas(root, width=400, height=400, bg='white')
canvas.pack()

# Variables to track the drawing state
last_x, last_y = None, None

# Functions to handle drawing
def start_draw(event):
    global last_x, last_y
    last_x, last_y = event.x, event.y

def draw(event):
    global last_x, last_y
    canvas.create_line((last_x, last_y, event.x, event.y), fill='black', width=5)
    last_x, last_y = event.x, event.y

def clear_canvas():
    canvas.delete("all")

# Bind the mouse events to the canvas
canvas.bind("<Button-1>", start_draw)
canvas.bind("<B1-Motion>", draw)

# Button to clear the canvas
clear_button = tk.Button(root, text="Clear", command=clear_canvas)
clear_button.pack()

# Function to capture and save the canvas content
capture_count =28
def capture_images():
    global capture_count
    if capture_count < 100:
        x = root.winfo_rootx() + canvas.winfo_x()
        y = root.winfo_rooty() + canvas.winfo_y()
        x1 = x + canvas.winfo_width()
        y1 = y + canvas.winfo_height()
        
        # Capture the canvas content
        im = ImageGrab.grab(bbox=(x, y, x1, y1))
        im.save(f"{images_folder}image_{capture_count}.png")
        print(f"We successfully saved: {capture_count}")
        capture_count=capture_count+1
        root.after(8000, capture_images)  # Schedule the next capture after 8 seconds

# Run the capture function after the canvas is set up with an 8-second delay for the first image
root.after(8000, capture_images)

# Run the tkinter main loop
root.mainloop()

We successfully saved: 28
We successfully saved: 29
We successfully saved: 30
We successfully saved: 31
We successfully saved: 32
We successfully saved: 33
We successfully saved: 34
We successfully saved: 35
We successfully saved: 36
We successfully saved: 37
We successfully saved: 38
We successfully saved: 39
We successfully saved: 40
We successfully saved: 41
We successfully saved: 42
We successfully saved: 43
We successfully saved: 44
We successfully saved: 45
We successfully saved: 46
We successfully saved: 47
We successfully saved: 48
We successfully saved: 49
We successfully saved: 50
We successfully saved: 51
We successfully saved: 52
We successfully saved: 53
We successfully saved: 54
We successfully saved: 55
We successfully saved: 56
We successfully saved: 57
We successfully saved: 58
We successfully saved: 59
We successfully saved: 60
We successfully saved: 61
We successfully saved: 62
We successfully saved: 63
We successfully saved: 64


In [32]:
import os
from PIL import Image

# Paths to the input and output folders
input_folder = 'captured_images/33/'
output_folder = 'captured_images/33/'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Target dimensions
target_width = 336
target_height = 279

# Function to crop the image to the target dimensions
def crop_to_target_size(image, target_width, target_height):
    width, height = image.size

    # Ensure the original image is large enough for the target size
    if width >= target_width and height >= target_height:
        left = (width - target_width) // 2
        upper = (height - target_height) // 2
        right = left + target_width
        lower = upper + target_height

        return image.crop((left, upper, right, lower))
    else:
        print(f"Image {image.filename} is too small for the target size.")
        return image  # Return the original image if it's too small

# Loop through each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(input_folder, filename)

        # Open the image
        image = Image.open(image_path)

        # Crop the image to the specified dimensions
        cropped_image = crop_to_target_size(image, target_width, target_height)

        # Save the cropped image to the output folder
        output_path = os.path.join(output_folder, filename)
        cropped_image.save(output_path)

        print(f"Cropped and saved {filename} to {output_folder}")

print("Cropping completed for all images.")


Cropped and saved image_29.png to captured_images/33/
Cropped and saved image_30.png to captured_images/33/
Cropped and saved image_31.png to captured_images/33/
Cropped and saved image_32.png to captured_images/33/
Cropped and saved image_33.png to captured_images/33/
Cropped and saved image_34.png to captured_images/33/
Cropped and saved image_35.png to captured_images/33/
Cropped and saved image_36.png to captured_images/33/
Cropped and saved image_37.png to captured_images/33/
Cropped and saved image_38.png to captured_images/33/
Cropped and saved image_39.png to captured_images/33/
Cropped and saved image_40.png to captured_images/33/
Cropped and saved image_41.png to captured_images/33/
Cropped and saved image_42.png to captured_images/33/
Cropped and saved image_43.png to captured_images/33/
Cropped and saved image_44.png to captured_images/33/
Cropped and saved image_45.png to captured_images/33/
Cropped and saved image_46.png to captured_images/33/
Cropped and saved image_47.p

In [ ]:
#Generate dataset
import cv2
import csv
import glob
  
header  =["label"]
for i in range(0,784):
   header.append("pixel"+str(i))
with open('dataset.csv', 'a') as f:
   writer = csv.writer(f)
   writer.writerow(header)
    
for label in range(10):
   dirList = glob.glob("captured_images/"+str(label)+"/*.png")
    
   for img_path in dirList:
       im= cv2.imread(img_path)
       im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
       im_gray = cv2.GaussianBlur(im_gray,(15,15), 0)
       roi= cv2.resize(im_gray,(28,28), interpolation=cv2.INTER_AREA)
        
       data=[]
       data.append(label)
       rows, cols = roi.shape
        
       ## Fill the data array with pixels one by one.
       for i in range(rows):
           for j in range(cols):
               k =roi[i,j]
               if k>100:
                   k=1
               else:
                   k=0
               data.append(k)
       with open('dataset.csv', 'a') as f:
           writer = csv.writer(f)
           writer.writerow(data)

In [13]:
! pip install cv2

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [15]:
!pip install opencv-python

In [6]:
!pip install pandas
!pip install scikit-learn

In [7]:
#Generate dataset
import cv2
import csv
import glob
  
header  =["label"]
for i in range(0,784):
   header.append("pixel"+str(i))
with open('dataset.csv', 'a') as f:
   writer = csv.writer(f)
   writer.writerow(header)
    
for label in range(10):
   dirList = glob.glob("captured_images/"+str(label)+"/*.png")
    
   for img_path in dirList:
       im= cv2.imread(img_path)
       im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
       im_gray = cv2.GaussianBlur(im_gray,(15,15), 0)
       roi= cv2.resize(im_gray,(28,28), interpolation=cv2.INTER_AREA)
        
       data=[]
       data.append(label)
       rows, cols = roi.shape
        
       ## Fill the data array with pixels one by one.
       for i in range(rows):
           for j in range(cols):
               k =roi[i,j]
               if k>100:
                   k=1
               else:
                   k=0
               data.append(k)
       with open('dataset.csv', 'a') as f:
           writer = csv.writer(f)
           writer.writerow(data)